In [16]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [17]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=options)

In [18]:
# Navigate to the website
url = "https://www.construction-europe.com/search?keyword=Liebherr"
driver.get(url)
time.sleep(2)

In [19]:
# Click the cookie button
driver.find_element(By.CSS_SELECTOR, '#cookieConsent > div > div > div > div.col-12.col-md-auto.text-center.py-2.align-self-center > button').click()

In [20]:
req = urllib.request.Request(url)
sourcecode = urllib.request.urlopen(url).read()
soup = BeautifulSoup(sourcecode, "html.parser")

In [21]:
# Click the options
brand = driver.find_element(By.ID, 'BrandFilter_6')
driver.execute_script("arguments[0].click();", brand)

type = driver.find_element(By.ID, 'TypeFilter_1')
driver.execute_script("arguments[0].click();", type)

category = driver.find_element(By.ID, 'CategoryFilter_4')
driver.execute_script("arguments[0].click();", category)

# Apply the selected options
driver.find_element(By.CSS_SELECTOR, '#filterOverlay > div > div > div > button.btn.btn-primary.btn-lg.btn-ApplyFilter').click()

In [22]:
# Scrape article details
full_address = []

for i in range(3):
    articles = soup.find_all("div", class_="khl-article-block-headline font-weight-bold h4")
    for article in articles:
        address = article.find("a")["href"]
        if address != 'articleurl' and address not in full_address:  # 'articleurl'이 아니고 중복되지 않은 경우에만 추가
            full_address.append(address)
        print(full_address)
        
    next_page_button = driver.find_element(By.CSS_SELECTOR, '#pagination > li.page-item.next > a').click()
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

full_address

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.InternationalCranes.media/news/still-time-to-book-for-itc-tower-crane-conference-in-barcelona/8028669.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.InternationalCranes.media/news/still-time-to-book-for-itc-tower-crane-conference-in-barcelona/8028669.article', 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.InternationalCranes.media/news/still-time-to-book-for-itc-tower-crane-conference-in-barcelona/8028669.article', 'https://www.construction-europe.com/news/heavy-wheeled-load

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article',
 'https://www.InternationalCranes.media/news/still-time-to-book-for-itc-tower-crane-conference-in-barcelona/8028669.article',
 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article',
 'https://www.InternationalCranes.media/news/is-construction-losing-out-on-efficiency-gains-by-using-too-many-tower-cranes-/8027936.article',
 'https://www.international-construction.com/news/greening-the-grey-how-are-new-concrete-technologies-tackling-decarbonisation-/8027861.article',
 'https://www.construction-europe.com/news/liebherr-reports-record-turnover-for-2022/8027981.article',
 'https://www.international-construction.com/news/is-the-impending-death-of-the-diesel-combustion-engine-exaggerated-/8026860.article',
 'https://www.construction-europe.com/news/fehmarnbelt-the-technology-behind-the-world-s-longest-immersed-tube-tun

In [23]:
article_data = []
for article_link in full_address:
    driver.get(article_link)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    release_date = ""
    title = ""
    content = ""
   
    # Scrape release date
    release_date_elem = soup.find("span", class_="PubDate")
    release_date = release_date_elem.text.strip() if release_date_elem else ""
            
    # Scrape news title
    title_elem = soup.find("h1", class_="khl-article-page-title")
    title = title_elem.text.strip() if title_elem else ""

    # Scrape article content
    first = soup.find("div", class_="col-12 khl-article-page-standfirst").text
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])

    # Check if content is empty
    if not content:
        continue

    # Append article data to the list
    article_data.append({"Date": release_date, "Title": title, "Content": content})
driver.quit()

In [24]:
# Create DataFrame from article data
df = pd.DataFrame(article_data)
df

,Date,Title,Content
0,30 May 2023,World’s top ten construction OEMs revealed,"The latest Yellow Table, International Constru..."
1,29 May 2023,Still time to book for ITC tower crane confere...,\n\n\nGet your tickets now for the Internation...
2,26 May 2023,Heavy wheeled loaders: The workhorse of the co...,Heavy wheeled loaders have a home in almost ev...
3,09 May 2023,Is construction losing out on efficiency gains...,\n\n\nA city skyline bristling with tower cran...
4,05 April 2023,Greening the grey: How are new concrete techno...,Construction relies heavily on concrete as a b...
5,04 April 2023,Liebherr reports record turnover for 2022,The German-Swiss company posted an annual turn...
6,17 March 2023,Is the impending death of the diesel combustio...,Whether it is the rush towards on-road electri...
7,09 March 2023,Fehmarnbelt: the technology behind the world’s...,Visible for miles around as clusters of orange...
8,01 March 2023,ConExpo-Con/Agg 2023: rolling news on the Las ...,Donaldson announces three new filtration produ...
9,01 March 2023,Global tower crane industry meets at ITC confe...,On 13 and 14 June the global tower crane indus...


In [25]:
# Save DataFrame to CSV
df.to_csv("liebherr_article_final.csv", index=False)